
# 🚀 Day 1: PySpark Fundamentals & Hands-On Prep

Welcome to Day 1 of PySpark Interview Prep!  
This notebook focuses on **DataFrame basics**, **transformations**, and **actions** — the foundation of Spark.


## ✅ Setup

In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, avg
spark = SparkSession.builder.appName("Day1-InterviewPrep").getOrCreate()


## 1️⃣ Create DataFrame from List

**Q1.** Create a PySpark DataFrame with the following data:

| Name   | Age | City           |
|--------|-----|----------------|
| Alice  | 29  | New York       |
| Bob    | 35  | San Francisco  |
| Cathy  | 25  | Seattle        |

- Show the DataFrame
- Print its schema

In [ ]:
# Your code here
df = spark.createDataFrame([('Alice', 29, 'New York'), ('Bob', 35, 'San Francisco'), ('Cathy', 25, 'Seattle')],["Name", "Age", "City"])

df.show()
df.printSchema()

## 2️⃣ Filter Rows by Age

**Q2.** Filter and return only rows where Age > 30.

In [ ]:
# Your code here
df_greater_than_30 = df.filter(col("Age")>30)
df_greater_than_30.show()

## 3️⃣ Add Column Based on Condition

**Q3.** Add a new column `IsAdult` where:
- True if Age >= 18
- False otherwise

In [ ]:
# Your code here
df = df.withColumn("IsAdult", when(df.Age>=18, True).otherwise(False))
df.show()

## 4️⃣ Group By and Aggregate

**Q4.** Using the following data, find average salary per department:

| Department | Employee | Salary |
|------------|----------|--------|
| HR         | John     | 4000   |
| HR         | Alice    | 3500   |
| IT         | Bob      | 4500   |
| IT         | Cathy    | 4800   |

In [ ]:
# Your code here
company_df = spark.createDataFrame([("HR", "John", 4000),("HR", "Alice", 3500),("IT", "Bob", 4500),("IT", "Cathy", 4800)],["Department", "Employee", "Salary"])

company_df.groupBy("Department").agg(avg("Salary")).show()

## 5️⃣ Read and Write Files

**Q5.** Read a CSV file and write it back as a Parquet file.

In [ ]:
# Your code here
emp_df = spark.read.format("csv").option("header", True).option("inferschema", True).load("./data/employee.csv")

emp_df.show()

emp_df.write.parquet("./data/employee.parquet", mode="overwrite")

## 🔍 Section 2: Column Operations & Expressions

## 6️⃣ Add Column: Name Length

**Q6.** Add a new column `NameLength` which stores the length of each name.

In [ ]:
# Your code here
from pyspark.sql.functions import length
df = df.withColumn("NameLength", length(df.Name))
df.show()

## 7️⃣ Add Column: Short City Code

**Q7.** Add a column `CityCode` with the first 3 letters of the city in uppercase.

In [ ]:
# Your code here
from pyspark.sql.functions import substring, upper
df = df.withColumn("CityCode", lit(upper(substring(df.City, 1, 3))))
df.show()

## 8️⃣ Use Regex & String Filters

**Q8.** Filter rows where City starts with 'S'.

In [ ]:
# Your code here
df.filter(df.City.like('S%')).show()

## 9️⃣ Drop Column

**Q9.** Drop the column `City`.

In [ ]:
# Your code here
df = df.drop(col("City"))
df.show()

## 🔄 Section 3: Sorting, Deduplication, Limiting

## 🔟 Sort by Age (Descending)

**Q10.** Sort the DataFrame by Age descending.

In [ ]:
# Your code here
df.orderBy(col("Age").desc()).show()

## 1️⃣1️⃣ Remove Duplicates

**Q11.** Drop duplicate records based on the Name column.

In [ ]:
# Your code here
df = df.dropDuplicates(["Name"])

## 🧼 Section 4: Null Handling

## 1️⃣2️⃣ Filter Null or Empty Department

**Q12.** Filter out rows where Department is null or empty.

In [ ]:
# Your code here
emp_df.filter(col("Department").isNull()).show()

## 1️⃣3️⃣ Fill Missing Values

**Q13.** Fill missing Department with `"Unknown"`.

In [ ]:
# Your code here
emp_df.fillna("Unknown", ("Department")).show()

## 🧪 Section 5: Type Casting & Schema

## 1️⃣4️⃣ Cast Salary from String to Integer

**Q14.** Create a DataFrame with Salary as string, then cast it to integer.

In [ ]:
# Your code here
from pyspark.sql.types import IntegerType

tst_df = spark.createDataFrame([("max", "2300"),("tom", "3100")],["Name", "Salary"])
tst_df.printSchema()

# convert salary to int
casted_tst_df = tst_df.select(col("Name"), col("Salary").cast(IntegerType()))
casted_tst_df.printSchema()

## 1️⃣5️⃣ Print Schema & Column Types

**Q15.** Use `.printSchema()` and `df.dtypes` to inspect types.

In [ ]:
# Your code here
df.printSchema()
emp_df.printSchema()
casted_tst_df.printSchema()

df.dtypes

## ⚙️ Section 6: Transformations vs Actions

## 🧠 Q16: What are Transformations and Actions?

📝 Spark separates operations into two types:
- **Transformations**: Lazy (e.g., `filter`, `select`, `groupBy`)
- **Actions**: Trigger computation (e.g., `show`, `collect`, `write`)

> Q: Why does nothing happen when you run `filter()` without `.show()`?

```markdown
# Write your explanation here
```
Transformations are lazy operations that define a new DataFrame (e.g. select, filter, withColumn). They don't trigger execution.

Actions are operations that trigger a Spark job and result in execution (e.g. show(), collect(), write()).

Example:
df.filter(col("age") > 30) does nothing until you call df.show() or df.write().

## 🧠 Q17: What is Lazy Evaluation?

> Q: Explain lazy evaluation in Spark and why it's beneficial.

```markdown
# Write your explanation here
```
Spark builds a logical execution plan and waits to execute until an action is called.

This allows Spark to optimize the entire job using techniques like pipelining, predicate pushdown, and logical plan pruning.

Benefits:

1. Efficient execution
2. Avoids unnecessary computation
3. Enables automatic optimizations


## 🧠 Q18: What are Wide vs Narrow Transformations?

> Q: Give an example of each and explain their difference in performance.

```markdown
# Write your explanation here
```
Narrow: No data shuffle, each partition is read independently
Examples: select, filter, map, withColumn

Wide: Requires shuffle across partitions (network heavy)
Examples: groupBy, join, distinct, repartition

Wide transformations can slow down jobs and require careful planning with partitioning and caching.

## 🧠 Q19: How Does Spark Read a CSV Differently Than Parquet?



```markdown
# Write your explanation here
```
#### CSV vs Parquet in Spark

| Feature        | CSV                          | Parquet                       |
|----------------|------------------------------|-------------------------------|
| Format         | Text-based                   | Binary, columnar              |
| Performance    | Slower (no compression/index)| Faster (compressed & indexed) |
| Schema         | Inferred or manually defined | Embedded within file          |
| Compression    | None or external             | Built-in (Snappy by default)  |
| Readability    | Human-readable               | Machine-efficient             |
| Use Case       | Simple ingestion, logs       | Analytics, reporting, large datasets |

> 💡 Use **Parquet** for optimized performance when dealing with large, structured data.


## 🧠 Q20: Explain Repartition vs Coalesce



```markdown
# Write your explanation here
```
#### `repartition()` vs `coalesce()` in Spark

| Feature          | `repartition()`                          | `coalesce()`                        |
|------------------|-------------------------------------------|-------------------------------------|
| Purpose          | Increase or redistribute partitions       | Reduce number of partitions         |
| Shuffle          | Yes (full shuffle of data)                | No (narrow transformation)          |
| Cost             | Expensive due to shuffling                | Efficient, avoids full shuffle      |
| Use Case         | After filtering or joins to rebalance     | Before writing to avoid small files |
| API Example      | `df.repartition(10)`                      | `df.coalesce(1)`                    |

> ✅ Use `repartition()` to increase partitions and balance load.  
> ✅ Use `coalesce()` to reduce output files efficiently.



# 🎯 Summary

In this notebook, we covered:
- Core DataFrame operations
- Aggregations and file IO
- Schema awareness
- Column and null handling
- Essential theory behind Spark’s execution

📌 Ready for Day 2? We'll dive into **joins**, **transformations**, and **interview-worthy questions** on data movement and optimization.
